# Tutorial 9: Seeing the Quarry
## Data Visualization with Matplotlib and Seaborn

---

### 📺 Video Walkthrough

Watch the video tutorial for this notebook: [YouTube Link](#)

---

### The Senator's Report

*"A chart is worth a thousand rows," Chief Archivist Mink said. "The senators don't read tables. They look at pictures. Show them a line going up, and they'll fund your expedition. Show them a line going down, and they'll ask why you're still employed."*

*The apprentice looked at the stack of reports. "So I should... draw pictures?"*

*"You should tell stories with data. The numbers are the truth. The chart is how you make them see it." Mink paused. "But never lie with a chart. The truth has a way of emerging from the Quarry, eventually. Usually with teeth."*

*—From the training of an apprentice archivist*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Create basic plots with matplotlib
2. Customize charts (titles, labels, colors)
3. Use seaborn for statistical visualizations
4. Create multi-panel figures
5. Choose the right chart for your data

By the end, you will be able to create clear, compelling visualizations that tell the story of the Quarry.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for nicer plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Load data from GitHub
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/yeller-quarry-data-science/main/data/"

catches = pd.read_csv(BASE_URL + "catches.csv")
creatures = pd.read_csv(BASE_URL + "creatures.csv")
crews = pd.read_csv(BASE_URL + "crews.csv")
prices = pd.read_csv(BASE_URL + "prices.csv")
incidents = pd.read_csv(BASE_URL + "incidents.csv")

# Prepare data
catches['date'] = pd.to_datetime(catches['date'])
catches['total_value'] = catches['quantity'] * catches['price_per_unit']
prices['date'] = pd.to_datetime(prices['date'])
incidents['date'] = pd.to_datetime(incidents['date'])

# Merge creature info into catches
catches = catches.merge(creatures[['creature_id', 'common_name', 'category', 'danger_rating']], on='creature_id')

print("Data loaded and prepared.")

---

## Part 1: Line Charts for Trends

Line charts show how values change over time.

In [ ]:
# Wharver prices over time
wharver = prices[prices['creature_id'] == 'CR009'].sort_values('date')

plt.figure(figsize=(10, 5))
plt.plot(wharver['date'], wharver['price_per_unit'], marker='o', linewidth=2, color='#e74c3c')
plt.title('Wharver Prices: The Rise of an Impossible Catch', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price per Unit')
plt.xticks(rotation=45)

# Add annotation for the spike
max_price = wharver['price_per_unit'].max()
max_date = wharver.loc[wharver['price_per_unit'].idxmax(), 'date']
plt.annotate('Live specimen captured!', 
             xy=(max_date, max_price),
             xytext=(max_date - pd.Timedelta(days=20), max_price - 50),
             arrowprops=dict(arrowstyle='->', color='black'),
             fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Multiple creatures on one chart
fig, ax = plt.subplots(figsize=(12, 6))

for creature_id, name in [('CR009', 'Wharver'), ('CR003', 'Stone Spine Lizard'), 
                          ('CR001', 'Leatherback Burrower')]:
    data = prices[prices['creature_id'] == creature_id].sort_values('date')
    ax.plot(data['date'], data['price_per_unit'], marker='o', label=name, linewidth=2)

ax.set_title('Creature Price Trends Across the Season', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price per Unit')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---

## Part 2: Bar Charts for Comparisons

In [ ]:
# Revenue by sector
sector_revenue = catches.groupby('sector')['total_value'].sum().sort_values(ascending=True)

plt.figure(figsize=(10, 6))
bars = plt.barh(sector_revenue.index, sector_revenue.values, color=sns.color_palette('viridis', len(sector_revenue)))
plt.xlabel('Total Revenue')
plt.title('Revenue by Sector: Where the Money Comes From', fontsize=14, fontweight='bold')

# Add value labels
for bar, value in zip(bars, sector_revenue.values):
    plt.text(value + 50, bar.get_y() + bar.get_height()/2, f'{value:,.0f}', 
             va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Catches by creature category
category_counts = catches.groupby('category').size().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(category_counts.index, category_counts.values, color=sns.color_palette('Set2'))
ax.set_xlabel('Creature Category')
ax.set_ylabel('Number of Catches')
ax.set_title('What Gets Caught: Catches by Creature Category', fontsize=14, fontweight='bold')

# Color the highest bar differently
bars[0].set_color('#e74c3c')

plt.tight_layout()
plt.show()

---

## Part 3: Histograms and Distributions

In [ ]:
# Distribution of catch values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(catches['total_value'], bins=20, color='steelblue', edgecolor='white')
axes[0].set_xlabel('Total Value')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Catch Values', fontsize=12, fontweight='bold')
axes[0].axvline(catches['total_value'].median(), color='red', linestyle='--', label=f'Median: {catches["total_value"].median():.0f}')
axes[0].legend()

# Log scale for better visibility
axes[1].hist(catches['total_value'], bins=20, color='steelblue', edgecolor='white')
axes[1].set_xlabel('Total Value')
axes[1].set_ylabel('Frequency (log scale)')
axes[1].set_title('Same Data, Log Scale', fontsize=12, fontweight='bold')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
# Distribution of danger ratings
plt.figure(figsize=(10, 5))
sns.histplot(data=creatures, x='danger_rating', bins=10, kde=True, color='#e74c3c')
plt.title('Danger Rating Distribution Across All Species', fontsize=14, fontweight='bold')
plt.xlabel('Danger Rating')
plt.ylabel('Count')
plt.show()

---

## Part 4: Scatter Plots for Relationships

In [ ]:
# Danger rating vs price
creature_stats = catches.groupby('creature_id').agg({
    'price_per_unit': 'mean',
    'danger_rating': 'first',
    'common_name': 'first'
}).reset_index()

plt.figure(figsize=(10, 6))
scatter = plt.scatter(creature_stats['danger_rating'], creature_stats['price_per_unit'], 
                     s=100, alpha=0.7, c=creature_stats['danger_rating'], cmap='YlOrRd')

# Label some interesting points
for _, row in creature_stats[creature_stats['price_per_unit'] > 400].iterrows():
    plt.annotate(row['common_name'], 
                (row['danger_rating'], row['price_per_unit']),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.xlabel('Danger Rating')
plt.ylabel('Average Price per Unit')
plt.title('Does Danger Pay? Price vs Danger Rating', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Danger Rating')
plt.tight_layout()
plt.show()

In [ ]:
# Seaborn regression plot
plt.figure(figsize=(10, 6))
sns.regplot(data=creature_stats, x='danger_rating', y='price_per_unit', 
           scatter_kws={'s': 100, 'alpha': 0.7},
           line_kws={'color': 'red'})
plt.title('Price vs Danger: With Trend Line', fontsize=14, fontweight='bold')
plt.xlabel('Danger Rating')
plt.ylabel('Average Price per Unit')
plt.show()

---

## Part 5: Box Plots for Distributions by Category

In [ ]:
# Catch values by creature category
plt.figure(figsize=(12, 6))
sns.boxplot(data=catches, x='category', y='total_value', palette='Set2')
plt.title('Catch Value Distribution by Category', fontsize=14, fontweight='bold')
plt.xlabel('Creature Category')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Violin plot shows distribution shape
plt.figure(figsize=(12, 6))
sns.violinplot(data=catches, x='category', y='total_value', palette='Set2')
plt.title('Catch Value Distributions: The Full Picture', fontsize=14, fontweight='bold')
plt.xlabel('Creature Category')
plt.ylabel('Total Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---

## Part 6: Heatmaps for Matrices

In [ ]:
# Create a pivot table for the heatmap
pivot = catches.pivot_table(
    index='sector',
    columns='category',
    values='total_value',
    aggfunc='sum',
    fill_value=0
)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot, annot=True, fmt='.0f', cmap='YlOrRd', linewidths=0.5)
plt.title('Revenue by Sector and Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap for creatures
creature_numeric = creatures[['metal_content_pct', 'danger_rating', 'avg_weight_kg', 'typical_depth_m']]
corr_matrix = creature_numeric.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
           square=True, linewidths=1)
plt.title('Creature Attribute Correlations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Part 7: Pie Charts (Use Sparingly)

In [ ]:
# Pie chart of incident types
incident_types = incidents['incident_type'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Basic pie
axes[0].pie(incident_types.values, labels=incident_types.index, autopct='%1.1f%%',
           colors=sns.color_palette('Set3'))
axes[0].set_title('Incident Types (Pie)', fontsize=12, fontweight='bold')

# Same data as bar (usually better)
axes[1].barh(incident_types.index, incident_types.values, color=sns.color_palette('Set3'))
axes[1].set_xlabel('Count')
axes[1].set_title('Incident Types (Bar)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

Note: Bar charts are usually clearer than pie charts, except when showing parts of a whole that sum to 100%.

---

## Part 8: Multi-Panel Figures

In [ ]:
# Create a dashboard-style figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Yeller Quarry Season Summary', fontsize=16, fontweight='bold')

# Panel 1: Revenue over time
monthly_revenue = catches.set_index('date')['total_value'].resample('M').sum()
axes[0, 0].bar(monthly_revenue.index.strftime('%B'), monthly_revenue.values, color='steelblue')
axes[0, 0].set_title('Monthly Revenue')
axes[0, 0].set_ylabel('Total Value')

# Panel 2: Catches by category
category_counts = catches['category'].value_counts()
axes[0, 1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
              colors=sns.color_palette('Set2'))
axes[0, 1].set_title('Catches by Category')

# Panel 3: Top 5 creatures by revenue
top_creatures = catches.groupby('common_name')['total_value'].sum().nlargest(5)
axes[1, 0].barh(top_creatures.index, top_creatures.values, color='coral')
axes[1, 0].set_xlabel('Total Revenue')
axes[1, 0].set_title('Top 5 Revenue-Generating Creatures')

# Panel 4: Incidents over time
monthly_incidents = incidents.set_index('date').resample('M').size()
axes[1, 1].plot(monthly_incidents.index, monthly_incidents.values, marker='o', 
               color='red', linewidth=2)
axes[1, 1].fill_between(monthly_incidents.index, monthly_incidents.values, alpha=0.3, color='red')
axes[1, 1].set_title('Incidents by Month')
axes[1, 1].set_ylabel('Number of Incidents')

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

---

## Part 9: Choosing the Right Chart

| Data Type | Best Chart |
|-----------|------------|
| Trend over time | Line chart |
| Compare categories | Bar chart |
| Show distribution | Histogram, box plot, violin plot |
| Show relationship | Scatter plot |
| Show parts of whole | Pie chart (or stacked bar) |
| Show matrix of values | Heatmap |
| Compare distributions across categories | Box plot, violin plot |

---

## ✏️ Exercises

### Exercise 1: The Senator's Report

Create a professional-looking line chart showing wharver, stone spine lizard, and yeller bird prices over time. Include a title, legend, and annotation marking the Redmane disaster (March 18).

In [ ]:
# Your code here



### Exercise 2: Sector Safety Analysis

Create a bar chart showing the number of incidents by sector, colored by severity. (Hint: you might want to use a stacked bar chart or grouped bars.)

In [ ]:
# Your code here



### Exercise 3: Crew Performance Dashboard

Create a 2x2 figure showing:
1. Total catches by crew
2. Total revenue by crew
3. Average catch value by crew
4. Number of incidents by crew

In [ ]:
# Your code here



### Exercise 4: The Yeller Pattern Visualization

Create a visualization that explores whether catches tend to come in "yeller numbers" (2, 3, 5, 7, 11). Compare the distribution of catch quantities to what you'd expect if quantities were random.

In [ ]:
# Your code here



### Exercise 5: Save Your Charts

Create a chart of your choice and save it as both PNG and PDF files using `plt.savefig()`.

In [ ]:
# Your code here



---

## 📋 Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Create figure | `plt.figure(figsize=(10, 6))` |
| Line chart | `plt.plot(x, y)` |
| Bar chart | `plt.bar(x, y)` / `plt.barh(x, y)` |
| Histogram | `plt.hist(data, bins=20)` |
| Scatter plot | `plt.scatter(x, y)` |
| Add title | `plt.title('Title')` |
| Add labels | `plt.xlabel()`, `plt.ylabel()` |
| Add legend | `plt.legend()` |
| Subplots | `fig, axes = plt.subplots(2, 2)` |
| Seaborn heatmap | `sns.heatmap(data)` |
| Seaborn boxplot | `sns.boxplot(data=df, x='a', y='b')` |
| Save figure | `plt.savefig('filename.png')` |

---

## ➡️ Next Tutorial

In **Tutorial 10: The Investigation**, you will bring together everything you've learned to conduct a capstone analysis—investigating the pattern of child disappearances near the Quarry boundary.

*"Three children," Mink said again, staring at the chart. "All in yeller numbers. All at the boundary. All between dusk and dawn."*

*"Coincidence?"*

*"In the Quarry?" She shook her head. "Nothing is coincidence in the Quarry. Everything has a pattern. We just have to find it."*

---

## 💾 Save Your Work

**File > Save a copy in Drive** to keep your completed notebook.